# GROMACS molecular dynamics demonstration

Molecular dynamics (MD) is a versatile method for simulating the time-dependent behavior of molecular systems. This notebook demonstrates how MD simulations can be run using the GROMACS software to study the self-assembly of lipids. To accelerate the simulations, a coarse-grained (CG) model using the Martini force field is employed. In addition, parallel processing with multiple CPU cores is leveraged to demonstrate the principles of high-performance computing (HPC).

Let's try inserting 50 dipalmitoylphosphatidylcholine (DPPC) lipids into a cubic simulation box with a side length of 6 Å. The resulting system will be stored in a file named `lipids.gro`.

In [ ]:
from ipywidgets import widgets
from IPython.display import display
box = widgets.FloatSlider(min=5, max=10, step=0.5, description='Box size')
nmol = widgets.IntSlider(min=10, max=100, description='# of lipids')
lip = widgets.Dropdown(options=['CPC',
                                'DLPC',
                                'DOPC',
                                'DPGS',
                                'DPPC',
                                'DPPG',
                                'DPSM',
                                'DTPC',
                                'DTPE',
                                'DTPG',
                                'DTPS',
                                'DXCE',
                                'DXPE',
                                'LPC',
                                'PADG',
                                'POPC',
                                'POPE',
                                'POPG',
                                'PPC',
                                'PRPC',
                                'PRPS',
                                'PVDG'],
                                description='Lipid')
display(lip)
display(nmol)
display(box)

In [ ]:
l = lip.value + '.gro'
n = nmol.value
b = box.value

In [ ]:
!gmx_mpi insert-molecules -o lipids -ci ff22/$l -nmol $n -try 200 -box $b

Next, we need to fill the `lipids.gro` system with water (solvate). The resulting system will be stored in the file `solv.gro`.

In [ ]:
!gmx_mpi solvate -cp lipids.gro -cs ff22/W400.gro -o solv -scale 2.0

Check the previous outputs for the number of lipids and water molecules that were *de facto* added. We need to modify a template topology file with these amounts, respectively.

In [ ]:
!sed "s|FIXME|$PWD|g" template/template.top > topol.top
!echo $lip.value 10 >> topol.top
!echo W 1060 >> topol.top

Now, let's copy a template recipe for performing *energy minimization*

In [ ]:
!cp template/em.mdp .
!gmx_mpi grompp -f em -c solv -p topol -o em -maxwarn 10

In [ ]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -v -s em

In [ ]:
!gmx_mpi make_ndx -f confout.gro < template/ndx_inp

In [ ]:
!cp template/md.mdp .
!sed -i "s/ADDINDEXGROUPS/$lip.value SOLVENTS/" md.mdp

In [ ]:
!gmx_mpi grompp -f md.mdp -c confout -p topol.top -maxwarn 10 -n index -o md

In [ ]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -deffnm md -v

In [ ]:
!echo 0 > trj_inp
!gmx_mpi trjconv -s md -f md.xtc -o md_whole.xtc -pbc mol < trj_inp

In [ ]:
import nglview as nv
import MDAnalysis as mda
import warnings
warnings.filterwarnings("ignore")

In [ ]:
u = mda.Universe('md.gro', 'md_whole.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lip.value)
view.add_point('W')
view

Cleanup

In [ ]:
rm -f *gro* *top* *xtc* *tpr* *log* *mdp* *cpt* *edr* *trr* *inp* *ndx*